# Plotting GOES Fire Detections using Google Earth Engine and Python

Earlier today, the Google Earth Engine team [posted a tutorial](https://medium.com/google-earth/how-to-generate-wildfire-boundary-maps-with-earth-engine-b38eadc97a38) to show how their platform can be used to process and visualize fire detections obtained by the GOES-16 and GOES-17 satellites.  This rough notebook expands their tutorial by demonstrating how this can be done from Python.

## Step 1: Import dependencies

First of all, we import the official Earth Engine package and use our Google Credentials to authenticate:

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

We will also be using the following packages:

In [ ]:
from datetime import datetime, timedelta
from pathlib import Path
import pytz
import requests
from skimage import io
from time import sleep

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, Video

## Step 2: Define the plotting area

We define the plotting area using a Polygon of lon/lat coordinates as follows:

In [ ]:
polygon = ee.Geometry.Polygon(
                [[[-122.54949658777271, 37.434613263197676],
                  [-122.54949658777271, 36.85224535996548],
                  [-121.78594678308521, 36.85224535996548],
                  [-121.78594678308521, 37.434613263197676]]], None, False)

We also define the plotting resolution here:

In [ ]:
scale = 100  # meters per pixel

## Step 3: Download image data from Earth Engine

We use the following helper functions to download a basemap and fire data:

In [ ]:
def download(url, filename):
    """Downloads a url to a given destination."""
    myfile = requests.get(url)
    open(filename, 'wb').write(myfile.content)
    return filename


def download_basemap(polygon, scale=500, filename=None):
    """Downloads a Landsat 8 basemap."""
    if filename is None:
        filename = f"basemap-scale{scale}.png"
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(polygon) \
        .filterDate('2017-01-01', '2021-01-01') \
        .sort('CLOUD_COVER', False)
    img = collection.mosaic().clip(polygon)
    params = {
      'bands': ['B4', 'B3', 'B2'],
      'min': 0,
      'max': 2000,
      'gamma': 1.5,
      'region': polygon,
      'scale': scale
    }
    url = img.getThumbURL(params)
    return download(url, filename)


def download_firemap(polygon, scale=500,
                     start='2020-08-20T00:00:00', end='2020-08-20T12:00:00',
                     filename=None):
    """Combines GOES-16 and GOES-17 into a single fire map.
    
    Follows the tutorial explained at:
    https://medium.com/google-earth/how-to-generate-wildfire-boundary-maps-with-earth-engine-b38eadc97a38
    """
    if filename is None:
        filename = f"firemap-scale{scale}-start{start}.png"
    print(f"Downloading {filename}")

    # Satellite data.
    goes_16_data = ee.ImageCollection('NOAA/GOES/16/FDCF') \
                        .filterDate(start, end).filterBounds(polygon)
    goes_17_data = ee.ImageCollection('NOAA/GOES/17/FDCF') \
                        .filterDate(start, end).filterBounds(polygon)
    
    # Conversion from mask codes to confidence values.
    fire_mask_codes = [10, 30, 11, 31, 12, 32, 13, 33, 14, 34, 15, 35];
    confidence_values = [1.0, 1.0, 0.9, 0.9, 0.8, 0.8, 0.5, 0.5, 0.3, 0.3, 0.1, 0.1];
    default_confidence_value = 0;

    def map_from_mask_codes_to_confidence_values(image):
        return image \
            .clip(polygon) \
            .remap(fire_mask_codes, confidence_values, default_confidence_value)
    
    goes_16_confidence = goes_16_data.select(['Mask']).map(map_from_mask_codes_to_confidence_values)
    goes_17_confidence = goes_17_data.select(['Mask']).map(map_from_mask_codes_to_confidence_values)
    
    goes_16_max_confidence = goes_16_confidence.reduce(ee.Reducer.max())
    goes_17_max_confidence = goes_17_confidence.reduce(ee.Reducer.max())

    affected_area_palette = ['white', 'yellow', 'orange', 'red', 'purple'];

    combined_confidence = ee.ImageCollection([goes_16_max_confidence, goes_17_max_confidence]) \
                                .reduce(ee.Reducer.min())
    
    kernel = ee.Kernel.square(2000, 'meters', True);
    smoothed_confidence = combined_confidence.reduceNeighborhood(
                            reducer=ee.Reducer.mean(),
                            kernel=kernel,
                            optimization='boxcar')
    smoothed_confidence = smoothed_confidence.updateMask(smoothed_confidence.gte(0.1))

    params = {
      'min': 0,
      'max': 1,
      'region': polygon,
      'scale': scale,
      'palette': affected_area_palette
    }

    url = smoothed_confidence.getThumbURL(params);
    return download(url, filename)

## Step 4: Download the basemap

We will plot the fire detections on top of a static base image, which we obtain here:

In [ ]:
basemap_fn = download_basemap(polygon, scale=scale)
print(f"Downloaded {basemap_fn}")

## Step 5: Download the fire data

In [ ]:
# Time of the first frame:
start = datetime.fromisoformat("2020-08-19T03:00:00")  # UTC

# Steps between frames:
stepsize = timedelta(hours=1)
# Number of frames:
steps = 60

# Now download the fire data for each frame
for idx in range(steps):
    center_time = start + idx*stepsize
    t1 = center_time - stepsize/2.
    t2 = center_time + stepsize/2.
    filename = f"firemap-{center_time}.png"
    download_firemap(polygon, scale=scale, start=t1, end=t2, filename=filename)
    sleep(1)

## Step 6: Blend the fire data with the base map

In [ ]:
# Define a helper function to blend the fire data with the base map
def blend(filename1, filename2):
    """Blends two PNG images together using their alpha channels."""
    im1 = io.imread(filename1) / 255.
    im2 = io.imread(filename2) / 255.
    alpha = im1[:,:,3] * 0.7
    blended = im2.copy()
    for band in range(3):
        blended[:,:,band] = alpha * im1[:,:,band] + (1. - alpha) * im2[:,:,band]
    return blended

# Blend all the frames with the base map
frames = []
filenames = sorted(list(Path('.').glob('firemap*png')))
for fn in filenames:
    b = blend(fn, basemap_fn)
    frames.append(b)

## Step 7: Combine the frames into a movie

In [ ]:
dpi = 50
scaling = 1
xsize = frames[0].shape[1] * scaling
ysize = frames[0].shape[0] * scaling

interval = 20000/30.

fig = plt.figure(figsize=(xsize/dpi, ysize/dpi), dpi=dpi, frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
plt.axis('off')
im = plt.imshow(frames[0])

timelabel = plt.text(1, 0.9, "Time",
                     fontsize=2000/dpi, color='white', horizontalalignment="right", weight=600,
                     bbox=dict(facecolor='black', edgecolor='none', alpha=0.5, pad=500/dpi),
                     transform=ax.transAxes)
plt.text(0, 0.07, "  GOES Fire Detections near Santa Cruz (Averaged)",
         fontsize=2000/dpi, color='white', weight=600,
         bbox=dict(facecolor='black', edgecolor='none', alpha=0.5, pad=500/dpi),
         transform=ax.transAxes)

def get_time(filename):
    utctimestamp = str(filename)[8:-4]
    d = pytz.utc.localize(datetime.fromisoformat(utctimestamp))
    d2 = d.astimezone(pytz.timezone("America/Los_Angeles"))
    return d2.strftime("%a, %b %-d, %-I %p  ")

def update_img(frameno):
    if frameno >= len(frames):
        return im
    im.set_data(frames[frameno])
    timelabel.set_text(get_time(filenames[frameno]))
    return im

ani = animation.FuncAnimation(fig, update_img, frames=len(frames)+5, interval=interval)
plt.close()

In [ ]:
display(HTML(ani.to_html5_video()))

In [ ]:
ani.save('goes-fire-detections.mp4', dpi=dpi*2)